# COGS 108 - Data Checkpoint
## Group: yippee!

# Names

- Mariana Villena
- Akanksha Pandey
- Rappel Ricafort
- Justin Lau
- Amaar Valliani

<a id='research_question'></a>
# Research Question

What are the key success factors for electric vehicle companies in terms of pricing and product features?

Subquestions:

* How has Tesla's success been affected by classical and/or emerging car companies' EV models?
* How have emerging EV companies entered the market and what success have they had?
* When did Classical car companies enter the EV market and how successful were they?

# Dataset(s)

- Dataset Name: Cheapest Electric Cars 2023
- Link to the dataset: [Kaggle Link](https://www.kaggle.com/datasets/kkhandekar/cheapest-electric-cars?resource=download)
- Number of observations: 308

This dataset features 308 EV models, their prices (in Germany), Acceleration, Range, Efficiency, Top Speed, type of drive, number of seats, and Fast Charge Speed. 

- Dataset Name: Stock Market Analysis of TESLA Inc from 2010 to 2022
- Link to the dataset: [Kaggle Link](https://www.kaggle.com/datasets/whenamancodes/tesla-inc-stock-market-analysis-founding-years)
- Number of observations: 3077

This dataset features the stock information of Tesla (the High and Low prices of the day, the opening, adjusted closing, and closing prices, the volume sold in 2010-2022). 

We will be finding the corresponding stock information of the companies listed in the Cheapest Electric Cars dataset and modeling how car features and price correspond to the stock performance of a company. Thus, we will add the company's average stock performance of each car model's year to a combined dataset. 

# Setup

In [51]:
import numpy as np
import pandas as pd

# Data Cleaning

We first downloaded the appropriate CSV files- one for each dataset described above. 
Then, we created the `cheapEVs` dataframe, which we cleaned by turning all string sections of measurements into floats or ints using the `dataToFloat()` function. 
We then created the `teslaStock` dataframe, which we cleaned by converting the date to datetime format, and creating an average daily stock price column, or `AvgHighLow`. Then, we averaged all the daily stock prices by the years available, stored in `year_avg_tesla_stock`. 

In [52]:
# read data of the cheapest electric cars sold in 2023 and check out the dataframe
cheapEVsPath = 'csvData/Cheapestelectriccars-EVDatabase_2023.csv'
cheapEVs = pd.read_csv(cheapEVsPath)
cheapEVs

,Name,Subtitle,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,PriceinGermany,PriceinUK
0,Lucid Air Dream Edition P,118 kWh useable battery Available sin...,2.7 sec,270 km/h,645 km,183 Wh/km,820 km/h,All Wheel Drive,5,"€218,000",NaN
1,Porsche Taycan Turbo S,83.7 kWh useable battery Available si...,2.8 sec,260 km/h,400 km,209 Wh/km,980 km/h,All Wheel Drive,4,"€189,668","£142,400"
2,Audi e-tron GT RS,85 kWh useable battery Available sinc...,3.3 sec,250 km/h,405 km,210 Wh/km,1000 km/h,All Wheel Drive,4,"€146,050","£115,000"
3,Renault Zoe ZE50 R110,52 kWh useable battery Available sinc...,11.4 sec,135 km/h,315 km,165 Wh/km,230 km/h,Front Wheel Drive,5,"€36,840",NaN
4,Audi Q4 e-tron 35,52 kWh useable battery Available sinc...,9.0 sec,160 km/h,285 km,182 Wh/km,360 km/h,Rear Wheel Drive,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
304,Volkswagen ID.7 Pro,77 kWh useable battery Expected from ...,8.0 sec,180 km/h,470 km,164 Wh/km,700 km/h,Rear Wheel Drive,5,"€55,000","£53,000"
305,Volkswagen ID.7 Pro S,86 kWh useable battery Expected from ...,7.0 sec,180 km/h,520 km,165 Wh/km,800 km/h,Rear Wheel Drive,5,"€60,000","£56,000"
306,Polestar 4 Long Range Single Motor,94 kWh useable battery Expected from ...,7.4 sec,180 km/h,510 km,184 Wh/km,760 km/h,Rear Wheel Drive,5,"€60,000","£55,000"
307,Polestar 4 Long Range Dual Motor,94 kWh useable battery Expected from ...,3.8 sec,200 km/h,475 km,198 Wh/km,710 km/h,All Wheel Drive,5,"€65,000","£60,000"


In [53]:
cheapEVs.describe()

,NumberofSeats
count,309.000000
mean,5.284790
std,0.978567
min,2.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,7.000000


In [54]:
# will quantify all numeric data into either a float or integer based on the type
def dataToNum(str_in):
    try:
        if(not isinstance(str_in, str)): # in case input is not a string
            return str_in
        
        str_in = str_in.lower()
        str_in = str_in.strip()

        if 'km/h' in str_in: # fast charge speed and top speed
            str_in = str_in.replace('km/h', '')
            output = int(str_in)
        elif 'wh/km' in str_in: # efficiency
            str_in = str_in.replace('wh/km', '')
            output = int(str_in)
        elif 'km' in str_in: # range
            str_in = str_in.replace('km', '')
            output = int(str_in)
        elif 'sec' in str_in: # acceleration
            str_in = str_in.replace(' sec', '')
            output = float(str_in)
        elif '€' in str_in: # price in Germany (euros)
            str_in = str_in.replace('€', '')
            str_in = str_in.replace(',','')
            output = int(str_in)
        elif '£' in str_in: #price in UK (pounds)
            str_in = str_in.replace('£', '')
            str_in = str_in.replace(',','')
            output = int(str_in)
        else: # handles null values
            output = np.nan
    except:
        output = np.nan     
    return output

In [55]:
cheapEVs["TopSpeed"] = cheapEVs["TopSpeed"].apply(dataToNum)
cheapEVs["FastChargeSpeed"] = cheapEVs["FastChargeSpeed"].apply(dataToNum)
cheapEVs['Acceleration'] = cheapEVs['Acceleration'].apply(dataToNum)
cheapEVs['Efficiency'] = cheapEVs['Efficiency'].apply(dataToNum)
cheapEVs['Range'] = cheapEVs['Range'].apply(dataToNum)
cheapEVs['PriceinGermany'] = cheapEVs['PriceinGermany'].apply(dataToNum)
cheapEVs['PriceinUK'] = cheapEVs['PriceinUK'].apply(dataToNum)

In [56]:
cheapEVs.Subtitle
#  there is lots of data in the Subtitle column; we can extract data on the battery, availability, and release date from this

0      118 kWh useable battery          Available sin...
1      83.7 kWh useable battery          Available si...
2      85 kWh useable battery          Available sinc...
3      52 kWh useable battery          Available sinc...
4      52 kWh useable battery          Available sinc...
                             ...                        
304    77 kWh useable battery          Expected from ...
305    86 kWh useable battery          Expected from ...
306    94 kWh useable battery          Expected from ...
307    94 kWh useable battery          Expected from ...
308    95 kWh useable battery          Expected from ...
Name: Subtitle, Length: 309, dtype: object

In [57]:
# function to extract the battery data from the Subtitle column
def batteryData(str_in):
    try:
        if(not isinstance(str_in, str)):
            return str_in
        
        str_in = str_in.lower()
        str_in = str_in.strip()

        if 'kwh' in str_in:
            str_in = str_in.partition('kwh')[0]
            output = float(str_in)
        else:
            output = np.nan
    except:
        output = np.nan     
    return output

In [58]:
cheapEVs['BatteryCapacity'] = cheapEVs['Subtitle'].apply(batteryData)

In [59]:
# function to extract the availability data from the Subtitle column
def availability(str_in):
    try:
        if(not isinstance(str_in, str)):
            return False
        
        str_in = str_in.lower()
        str_in = str_in.strip()

        if 'available' in str_in:
            output = True
        else:
            output = False
    except:
        output = False     
    return output

In [60]:
cheapEVs['Available'] = cheapEVs['Subtitle'].apply(availability)

In [61]:
# function to extract the release date data from the Subtitle column
def release(str_in):
    try:
        if(not isinstance(str_in, str)):
            return ""
        
        str_in = str_in.lower()
        str_in = str_in.strip()

        str_in = str_in.split(' ')
        month = str_in[-2]
        year = str_in[-1]
        return (month +" "+ year)
    except:
        output = ""
    return output

In [62]:
cheapEVs['Release'] = cheapEVs['Subtitle'].apply(release)
# now that we have extracted the useful data from the column, we can drop it
cheapEVs = cheapEVs.drop(columns=['Subtitle'])
cheapEVs

,Name,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,PriceinGermany,PriceinUK,BatteryCapacity,Available,Release
0,Lucid Air Dream Edition P,2.7,270,645,183,820.0,All Wheel Drive,5,218000.0,NaN,118.0,True,january 2022
1,Porsche Taycan Turbo S,2.8,260,400,209,980.0,All Wheel Drive,4,189668.0,142400.0,83.7,True,january 2020
2,Audi e-tron GT RS,3.3,250,405,210,1000.0,All Wheel Drive,4,146050.0,115000.0,85.0,True,march 2021
3,Renault Zoe ZE50 R110,11.4,135,315,165,230.0,Front Wheel Drive,5,36840.0,NaN,52.0,True,november 2019
4,Audi Q4 e-tron 35,9.0,160,285,182,360.0,Rear Wheel Drive,5,NaN,NaN,52.0,True,june 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Volkswagen ID.7 Pro,8.0,180,470,164,700.0,Rear Wheel Drive,5,55000.0,53000.0,77.0,False,october 2023
305,Volkswagen ID.7 Pro S,7.0,180,520,165,800.0,Rear Wheel Drive,5,60000.0,56000.0,86.0,False,october 2023
306,Polestar 4 Long Range Single Motor,7.4,180,510,184,760.0,Rear Wheel Drive,5,60000.0,55000.0,94.0,False,february 2024
307,Polestar 4 Long Range Dual Motor,3.8,200,475,198,710.0,All Wheel Drive,5,65000.0,60000.0,94.0,False,february 2024


In [63]:
cheapEVs.describe()

,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,NumberofSeats,PriceinGermany,PriceinUK,BatteryCapacity
count,309.000000,309.000000,309.000000,309.000000,306.000000,309.000000,282.000000,198.000000,309.000000
mean,7.532039,180.754045,361.440129,199.430421,543.562092,5.284790,71039.826241,64255.196970,71.076052
std,3.127109,38.738187,110.861212,33.021626,247.200836,0.978567,41731.536969,37481.721471,21.139409
min,2.100000,125.000000,95.000000,150.000000,150.000000,2.000000,21940.000000,25995.000000,16.700000
25%,4.900000,150.000000,275.000000,174.000000,330.000000,5.000000,46990.000000,38876.250000,56.000000
50%,7.000000,180.000000,370.000000,192.000000,500.000000,5.000000,58752.500000,52682.500000,74.000000
75%,9.000000,200.000000,440.000000,214.000000,680.000000,5.000000,77275.000000,76336.500000,85.000000
max,19.100000,320.000000,685.000000,295.000000,1240.000000,7.000000,400000.000000,350000.000000,123.000000


In [64]:
# Tesla Stock Market Analysis Dataset
teslaStockPath = "csvData/TSLAstock.csv"
teslaStock = pd.read_csv(teslaStockPath)
teslaStock

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
1,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
2,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
3,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
4,2010-07-07,1.093333,1.108667,0.998667,1.053333,1.053333,103825500
...,...,...,...,...,...,...,...
3072,2022-09-13,292.899994,297.399994,290.399994,292.130005,292.130005,68229600
3073,2022-09-14,292.239990,306.000000,291.640015,302.609985,302.609985,72628700
3074,2022-09-15,301.829987,309.119995,300.720001,303.750000,303.750000,64795500
3075,2022-09-16,299.609985,303.709991,295.600006,303.350006,303.350006,86949500


In [65]:
teslaStock.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3077.000000,3077.000000,3077.000000,3077.000000,3077.000000,3.077000e+03
mean,55.502174,56.748047,54.161176,55.495537,55.495537,9.340392e+07
std,93.913081,96.085026,91.517219,93.851345,93.851345,8.235816e+07
min,1.076000,1.108667,0.998667,1.053333,1.053333,1.777500e+06
25%,8.192667,8.354667,7.970667,8.113333,8.113333,4.144350e+07
50%,15.983333,16.242001,15.687333,16.000668,16.000668,7.541550e+07
75%,23.586666,23.916668,23.229334,23.523333,23.523333,1.173030e+08
max,411.470001,414.496674,405.666656,409.970001,409.970001,9.140820e+08


In [66]:
# create a column of average daily stock price by averaging the high and low stock prices
teslaStock["AvgHighLow"] = (teslaStock['High']+teslaStock['Low'])/2
teslaStock["AvgHighLow"]

0         1.790666
1         1.539667
2         1.393667
3         1.194333
4         1.053667
           ...    
3072    293.899994
3073    298.820007
3074    304.919998
3075    299.654999
3076    303.819992
Name: AvgHighLow, Length: 3077, dtype: float64

In [67]:
# convert date column to datetime 
teslaStock['Date'] = pd.to_datetime(teslaStock['Date'])
teslaStock['Date']

0      2010-06-30
1      2010-07-01
2      2010-07-02
3      2010-07-06
4      2010-07-07
          ...    
3072   2022-09-13
3073   2022-09-14
3074   2022-09-15
3075   2022-09-16
3076   2022-09-19
Name: Date, Length: 3077, dtype: datetime64[ns]

In [68]:
#resample by year and calculate average stock price of every year
year_avg_tesla_stock = teslaStock.groupby(teslaStock.Date.dt.year)['AvgHighLow'].mean()
year_avg_tesla_stock[2010]

1.558524534883721